## Group Project 509 ,  Hoori Javadnia & Fatime Khosravi

# descriptive statistics on your final-project data

In [126]:
pip install --upgrade emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.3/356.3 kB 2.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.5.1-py2.py3-none-any.whl size=351210 sha256=f3fd1a46bf48aac5b7534d8cd0c047593aa45ce1cf8355c3101acb1d9eb87e1c
  Stored in directory: /Users/javadniahoori/Library/Caches/pip/wheels/a0/bb/4c/96ea18d2d9a98bc7f3a6559c8bce0f72601b375af92e3b43a0
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.2.0
    Uninstalling emoji-2.2.0:
      Successfully uninstalled emoji-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 3.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [127]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import time
import random
from collections import OrderedDict
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk
import random


py.init_notebook_mode(connected=True)


from nltk.stem import WordNetLemmatizer
from random import randint
from textblob import TextBlob
from time import sleep

color = sns.color_palette()
%matplotlib inline


from collections import Counter, defaultdict
from string import punctuation
from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix
import plotly.io as pio

color = sns.color_palette()
%matplotlib inline
import emoji

In [128]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/javadniahoori/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
import os
import re
import emoji
import pandas as pd
import numpy as np

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

sw = stopwords.words("english")

In [130]:

  import re
re.compile('<title>(.*)</title>')

re.compile(r'<title>(.*)</title>', re.UNICODE)

In [131]:
from nltk.tokenize import word_tokenize

## Read the data 

In [132]:
import pandas as pd
data = pd.read_csv('/Users/javadniahoori/Downloads/509/group/cus_q_dataset.csv')

In [133]:
print(data.shape)
data.head(10)

(19846, 2)


,Query,Class
0,What is the PDF document stating my final tran...,Billing
1,\r\n\r\nI noticed that I was charged an additi...,Billing
2,I charged 83.00 a payment on my card but still...,Billing
3,\r\n\r\nWhat address should I use to send this...,Billing
4,\r\n\r\nI noticed an pending charge on my cell...,Billing
5,"\r\n\r\nHi there, I noticed on my last stateme...",Billing
6,Has my credit card payment of \$50 been proces...,Billing
7,\r\n\r\nI typically pay my bill within busines...,Billing
8,\r\n\r\nWere curious what billing plan of ours...,Billing
9,\r\n\r\nCan you please clarify the amount I am...,Billing


In [134]:
#Summary of the dataset
data.describe()

,Query,Class
count,19844,19846
unique,19689,4
top,\r\n\r\nMy laptop won't turn on when I press t...,Billing
freq,8,4963


In [135]:
print(data.shape)
data.head(10)

(19846, 2)


,Query,Class
0,What is the PDF document stating my final tran...,Billing
1,\r\n\r\nI noticed that I was charged an additi...,Billing
2,I charged 83.00 a payment on my card but still...,Billing
3,\r\n\r\nWhat address should I use to send this...,Billing
4,\r\n\r\nI noticed an pending charge on my cell...,Billing
5,"\r\n\r\nHi there, I noticed on my last stateme...",Billing
6,Has my credit card payment of \$50 been proces...,Billing
7,\r\n\r\nI typically pay my bill within busines...,Billing
8,\r\n\r\nWere curious what billing plan of ours...,Billing
9,\r\n\r\nCan you please clarify the amount I am...,Billing


In [136]:
data.columns

Index(['Query', 'Class'], dtype='object')

In [137]:
# Stopwords
sw = stopwords.words("english")

# Some punctuation variations
punct = list(set(punctuation))
punct.append(",")
punct = "".join(punct)
tw_punct = list(set(punct) - set(["#"]))
tw_punct = "".join(tw_punct)

In [145]:
import emoji.unicode_codes


In [148]:
# Place any addtional functions or constants you need here. 
def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table', 'table style="display:inline"'), 
        raw = True
    )

# Convert lyric text file to dataframe
def txt_to_dict(artist):
    # Create list of files in artist's subdirectory
    song_list = os.listdir(lyrics_path + artist)
    # Create empty list to hold data
    lyrics_list = []
    # Read each file in the artist's folder
    for song in song_list:
        metadata = {}
        path_to_song = lyrics_path + artist + "/" + song
        with open(path_to_song, "r") as song_lyrics:
            lyrics = song_lyrics.read()
        metadata["Artist"] = artist
        metadata["Title"] = lyrics.split("\n", 2)[0]
        metadata["Lyrics"] = lyrics.split("\n", 2)[2]
        # Append metadata to lyrics_list
        lyrics_list.append(metadata)
    # Convert lyrics_list to dict
    lyrics_df = pd.DataFrame.from_records(lyrics_list)
    
    return lyrics_df

# Global text prep pipeline function
def prepare(text, pipeline): 
    tokens = str(text)

    for transform in pipeline: 
        tokens = transform(tokens)
        
    return(tokens)
    
# Remove Punctuation Function
def remove_punctuation(text): 
    text = text.translate(str.maketrans("", "", tw_punct))
    return text

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# Remove White Spaces Function
def remove_whitespace(text):
    text = whitespace_pattern.sub(r" ", text)
    return text

def tokenize(text):  
    tokens = text.split(" ")
    # Remove empty splits
    tokens = [x for x in tokens if x != ""]
    return tokens

def remove_stop(tokens):
    tokens = [x for x in tokens if not x in sw]
    return tokens



def descriptive_stats(df_col, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """
    # Initialize blank counter
    token_counter = Counter()
    # Update counter row by row with token column
    df_col.map(token_counter.update)
    
    num_tokens = sum(token_counter.values())
    num_unique_tokens = len(token_counter)
    lexical_diversity = num_unique_tokens / num_tokens

    num_characters = 0
    for k,v in token_counter.items():
        token_length = len(k) * v
        num_characters += token_length

    top_tokens = token_counter.most_common(5)
    # Convert top 5 tokens to dataframe
    top_token_df = pd.DataFrame(top_tokens, columns = ["Token", "Frequency"])
    
    if verbose == True:        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        print("The top five most common tokens are:")
        print(top_token_df)
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])

In [149]:
# Apply Processing Pipeline
prep_pipeline = [str.lower, remove_punctuation, 
                   remove_whitespace, tokenize, 
                   remove_stop]

data["tokens"] = data["Query"].apply(
    prepare, pipeline = prep_pipeline
)


In [150]:
data.head()

,Query,Class,tokens
0,What is the PDF document stating my final tran...,Billing,"[pdf, document, stating, final, transaction, b..."
1,\r\n\r\nI noticed that I was charged an additi...,Billing,"[noticed, charged, additional, fee, even, thou..."
2,I charged 83.00 a payment on my card but still...,Billing,"[charged, 8300, payment, card, still, seen, cr..."
3,\r\n\r\nWhat address should I use to send this...,Billing,"[address, use, send, customers, invoice, recen..."
4,\r\n\r\nI noticed an pending charge on my cell...,Billing,"[noticed, pending, charge, cellular, phone, bi..."


In [125]:
data["num_tokens"] = data["tokens"].map(len)

In [151]:
print("Descriptive Stats:\n")
print(descriptive_stats(data["tokens"]))


Descriptive Stats:

There are 178929 tokens in the data.
There are 13754 unique tokens in the data.
There are 1128840 characters in the data.
The lexical diversity is 0.077 in the data.
The top five most common tokens are:
       Token  Frequency
0     return       4090
1    product       3173
2       item       2682
3  purchased       2012
4   recently       1819
[178929, 13754, 0.07686847855853439, 1128840]


In [ ]:
# Save the preprocessed data to a new CSV file
#data.to_csv('preprocessed_data.csv', index=False)